In [1]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter
import mpmath
from fractions import Fraction
from math import floor
import pickle
import random

from scipy.optimize import linprog

It is possible to change the set of assortments, and therefore the constraints.

In [7]:
# compute the subsets of cardinality at most k
def generate_subsets_up_to_k(elements, k):
    """Generate a list of the subsets of elements, of cardinality at most k

    Args:
        elements (list): list of the products which are still in stock
        k (int): cardinality constraint

    Returns:
        list: list of the subsets of elements, of cardinality at most k
    """
    subsets = []
    n = len(elements) # number of products which are still in stock
    for i in range(1 << n):  # Nombre de sous-ensembles possibles = 2^n
        subset = [elements[j] for j in range(n) if (i & (1 << j))]
        if len(subset) <= k and len(subset) >= 1:
            subsets.append(subset)
    return subsets

# compute the feasible assortments
def set_assortments(l,k):
    """Return the list of the feasible assortments

    Args:
        l (list of int): current stock
        k (int): cardinality constraint

    Returns:
        list: list of the feasible assortments
    """
    N_assortment = [i for i in range(len(l)) if l[i] > 0] #list of the products which are still in stock
    return generate_subsets_up_to_k(N_assortment, k)


# Optimal solution

Put $k = len(l)$ for having the unconstrained problem.

In [2]:
# compute E[T_{l1,..,lN}] given the assortment offered at the first step
def T(l, S, list_v, k, memory, assortment):
    """Compute the total expected number of customer required to clear the stock,
    for a policy who offers first the assortment S until a product is selected by a customer, then it follows the optimal policy. 

    Args:
        l (list of integers): the current stock
        S (set): the first assortment offered to the customers, until a product is selected by a customer
        list_v (list): list of the preference weights, without the no-purchase
        k (int): cardinality constraint
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.

    Returns:
        float: the total expected time
    """
    I_N = np.identity(len(list_v)) #identity matrix
    T = Fraction(1,1) #initialization of T
    V_S = Fraction(0,1) #Initialization of V(S)
    for i in S :
        V_S = V_S + list_v[i] # V(S) update
        if tuple(l - I_N[i]) in memory : # if the state l - e_i was already computed, we use this value
            Ti = memory[tuple(l - I_N[i])]
        else : #otherwise, we do the computation
            Ti = f(l - I_N[i],k, list_v, memory, assortment)[0]
        T = T + list_v[i] *Ti # Update of T
    return T/V_S + Fraction(1,1) 

# compute the subsets of cardinality at most k
def generate_subsets_up_to_k(elements, k):
    """Generate a list of the subsets of elements, of cardinality at most k

    Args:
        elements (list): list of the products which are still in stock
        k (int): cardinality constraint

    Returns:
        list: list of the subsets of elements, of cardinality at most k
    """
    subsets = []
    n = len(elements) # number of products which are still in stock
    for i in range(1 << n):  # Nombre de sous-ensembles possibles = 2^n
        subset = [elements[j] for j in range(n) if (i & (1 << j))]
        if len(subset) <= k and len(subset) >= 1:
            subsets.append(subset)
    return subsets

# compute the feasible assortments
def set_assortments(l,k):
    """Return the list of the feasible assortments

    Args:
        l (list of int): current stock
        k (int): cardinality constraint

    Returns:
        list: list of the feasible assortments
    """
    N_assortment = [i for i in range(len(l)) if l[i] > 0] #list of the products which are still in stock
    return generate_subsets_up_to_k(N_assortment, k)

# compute the assortment corresponding to the minimum, and the value of the minimum
def f(l, k, list_v, memory, assortment) :
    """Return the optimal value, after exploring all the feasible solutions.

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    Returns:
        float or Fraction: optimal value
        list : optimal assortment
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    """
    if tuple(l) in memory : # If it was already computed, we return this value
        return memory[tuple(l)], assortment[tuple(l)], memory, assortment
    S_set = set_assortments(l,k) #Compute the list of the feasible assortments
    S_min = []
    T_min = float('inf')
    for S in S_set : #Exploring all the feasible solutions
        T_s = T(l, S, list_v, k, memory, assortment)
        if T_s < T_min: #Updating the optimal assortment and the optimal value
            T_min = T_s
            S_min = S
    assortment[tuple(l)] = S_min #Updating the dictionaries
    memory[tuple(l)] = T_min
    return T_min, S_min, memory, assortment

def solution_opt(l,k, list_v, memory, assortment):
    """Return the dictionaries corresponding to the optimal solution

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        assortment (dictionary): dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock

    Returns:
        dictionary : dictionary, where the keys are the states of the stock
        dictionary: dictionary, where the keys are the states of the stock, and the values are the optimal assortment correponding to that stock.
    """
    N = len(l)
    if tuple([0 for i in range(N)]) not in memory : #initialization of the dictionaries
        memory[tuple([0 for i in range(N)])] = Fraction(0,1)
        assortment[tuple([0 for i in range(N)])] = []
    if np.sum(l) == 0 :
        return Fraction(0,1),[]
    return f(l,k,list_v, memory, assortment)[2:4]

In [3]:
memory = {}
assortment = {}
list_v = [Fraction(1,1), Fraction(4,1), Fraction(5,1)]

In [4]:
# If you modify the preference weights, you need to reset the dictionaries
list_l = [10,10,10]
memory, assortment = solution_opt(list_l,2,list_v, memory, assortment)

In [5]:
memory[1,1,1]

Fraction(737, 180)

# Fluid approximation

In [12]:
def fluid_approx(list_l,list_v, k):
    """Compute the fluid approximation

    Args:
        list_l (list of int): list of the inventory
        list_v (list): list of the preference weights, without the no-purchase
        k (int): cardinality constraint

    Returns:
       dictionary : a dictionary of the information about the solution. See the documentation about the linprog function.
       solution.x = list of the optimal variables, solution.fun = value of the objective function
       dictionary : dictionary of the probability, omega*
       float : value of the objective function
    """
    S_0k = set_assortments(list_l, k)
    A = np.zeros((len(list_v),len(S_0k)))
    for r in range(len(S_0k)):
        v = 1/(1 + np.sum(np.array([list_v[y] for y in S_0k[r]])))
        for i in S_0k[r]:
            A[i,r] = list_v[i] * v
    bounds = [(0, None)] * len(S_0k) #variables
    b = np.array(list_l)
    c = np.array([1 for i in range(len(S_0k))])
    result = linprog(c, A_ub=-A, b_ub= -b, bounds=bounds, method='highs')
    proba = {}
    for i in range(len(S_0k)):
        proba[tuple(S_0k[i])] = result.x[i]/result.fun
    return result, proba, result.fun

# Approximation policy

In [56]:
# compute the subsets of cardinality at most k
def generate_subsets_up_to_k(elements, k):
    """Generate a list of the subsets of elements, of cardinality at most k

    Args:
        elements (list): list of the products which are still in stock
        k (int): cardinality constraint

    Returns:
        list: list of the subsets of elements, of cardinality at most k
    """
    subsets = []
    n = len(elements) # number of products which are still in stock
    for i in range(1 << n):  # Nombre de sous-ensembles possibles = 2^n
        subset = [elements[j] for j in range(n) if (i & (1 << j))]
        if len(subset) <= k and len(subset) >= 1:
            subsets.append(subset)
    return subsets

# compute the feasible assortments
def set_assortments(l,k):
    """Return the list of the feasible assortments

    Args:
        l (list of int): current stock
        k (int): cardinality constraint

    Returns:
        list: list of the feasible assortments
    """
    N_assortment = [i for i in range(len(l))]
    return generate_subsets_up_to_k(N_assortment, k)

# compute the assortment corresponding to the minimum, and the value of the minimum
def f_approx(l, k, list_v, memory, proba) :
    """ Return the expected number of customers to clear the stock for the approximation policy

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        list_v (list): list of the preference weights, without the no-purchase
        memory (dictionary): dictionary, where the keys are the states of the stock
        proba (dictionary) : dictionary, where the  keys are the feasible assortments, and the values are dictionaries,
        whose keys are elements of the feasible assortment and the values are the probability of choosing this element knowing that you show this assortment

    Returns:
        float: the expected number of customers to clear the stock for the approximation policy
        dictionary : updated dictionary of the expected number of customers
    """
    if tuple(l) in memory :
        return memory[tuple(l)], memory
    S_set = set_assortments(l,k)
    res = Fraction(0,1)
    base = Fraction(0,1)
    N_assortment = [i for i in range(len(l)) if l[i] > 0]
    for i in N_assortment :
        intermed = Fraction(0,1)
        for S in S_set :
            if i in S:
                intermed = intermed + list_v[i]/(1+ np.sum(np.array([list_v[r] for r in S])))*proba[tuple(S)]
        tup = [l[j] for j in range(len(l))]
        tup[i] = tup[i] - 1
        res = res + f_approx(tup,k,list_v, memory, proba)[0] * intermed
        base = base + intermed
    res = (res + 1) / base
    memory[tuple(l)] = res
    return res, memory

def solution_approx(l,k, list_v, memory):
    """Return the dictionariy corresponding to the optimal solution

    Args:
        l (list of integers): the current stock
        k (int): cardinality constraint
        memory (dictionary): dictionary, where the keys are the states of the stock, and the values the optimal values corresponding to that stock
        
    Returns:
        dictionary : dictionary, where the keys are the states of the stock
    """
    N = len(l)
    if tuple([0 for i in range(N)]) not in memory : #initialization of the dictionaries
        memory[tuple([0 for i in range(N)])] = Fraction(0,1)
    if np.sum(l) == 0 :
        return Fraction(0,1)
    proba = fluid_approx(l,list_v,k) [1]
    return f_approx(l,k,list_v, memory, proba)[1]

In [57]:
memory = {}
list_v = [Fraction(1,1), Fraction(4,1), Fraction(5,1)]

In [58]:
list_l = [10,10,10]
memory = solution_approx(list_l,2,list_v, memory)

In [59]:
memory[tuple(list_l)]/fluid_approx(list_l,list_v,2)[0].fun

1.273414073229726

# The case of an uniform stock

In [ ]:
dic = {}
def K(x):
    if np.sum(x) ==0:
        dic[tuple(x)] = 0.
        return 0, dic
    if tuple(x) in dic :
        return dic[tuple(x)], dic
    tab = [i for i in range(len(x)) if x[i]>0]
    res = 0.
    for i in tab:
        xi = [x[j] for j in range (len(x))]
        xi[i] = xi[i] - 1
        res = res + K(xi)[0]
    dic[tuple(x)] = (1 + res)/len(tab)
    return (1 + res)/len(tab), dic

In [ ]:
N = 10
tab =np.zeros(N-1)
for i in range(1,N):
    tab[i-1] = (K([i,i])[0]/i - 1)*2
listx = np.array([i for i in range(1,N)])
#listf = 1/listx**0.5 + 0.1/listx**0.4 -1/(3*listx**np.exp(1))
listf = 1/listx**0.5
plt.figure()
plt.plot(listx, tab, color = 'r', label = 'K')
plt.plot(listx, listf)
#plt.plot(listx, (tab - listf), color = 'g')
plt.legend()
plt.show()